In [1]:
import pandas as pd

In [2]:
df_fusionné = pd.read_csv('df_fusionné.csv')

In [3]:
df_fusionné_binaire = df_fusionné.sample(frac=0.2)
df_fusionné_binaire.iloc[:, 0] = df_fusionné_binaire.iloc[:, 0].apply(lambda x: 0 if x == 'N' else 1)

In [4]:
# Count the occurrences of each class
counts = df_fusionné_binaire.iloc[:, 0].value_counts()

# Find the number of instances in the minority class
min_count = counts.min()

# Create balanced DataFrame through undersampling
df_balanced_under = pd.concat([
    df_fusionné_binaire[df_fusionné_binaire.iloc[:, 0] == 0].sample(min_count),
    df_fusionné_binaire[df_fusionné_binaire.iloc[:, 0] == 1].sample(min_count)
])

# Shuffle the DataFrame to mix the classes
df_balanced_under = df_balanced_under.sample(frac=1)


In [5]:
# Count occurrences of 0 and 1 in the first column
value_counts = df_balanced_under.iloc[:, 0].value_counts()

# Display the counts
print(value_counts)

ColumnName
1    7610
0    7610
Name: count, dtype: int64


In [6]:
df_balanced_under

,ColumnName,0,1,2,3,4,5,6,7,8,...,350,351,352,353,354,355,356,357,358,359
20483,1,0.508545,0.505895,0.503702,0.501856,0.500906,0.500394,0.500283,0.500372,0.499971,...,0.565102,0.568349,0.549473,0.523156,0.520707,0.524836,0.520211,0.514226,0.512134,0.511084
163574,0,0.497136,0.502909,0.509403,0.515139,0.516254,0.515343,0.513249,0.510358,0.508750,...,0.471170,0.472104,0.471517,0.470890,0.469555,0.468742,0.472643,0.478504,0.484460,0.490846
117723,0,0.452258,0.451674,0.450764,0.449496,0.449616,0.450155,0.450601,0.451089,0.451420,...,0.454003,0.454124,0.450000,0.445048,0.438331,0.432121,0.433860,0.438348,0.444472,0.451010
188456,1,0.464646,0.463487,0.462395,0.461267,0.460037,0.458851,0.457593,0.456443,0.455995,...,0.474899,0.471980,0.470250,0.469086,0.468061,0.467271,0.466662,0.466133,0.465840,0.465559
31618,1,0.493794,0.492482,0.485528,0.490477,0.487291,0.484120,0.486129,0.489573,0.488875,...,0.627442,0.619136,0.613893,0.608644,0.604008,0.598585,0.584290,0.563741,0.554028,0.556172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99791,0,0.503959,0.504939,0.505964,0.507032,0.508004,0.508925,0.509869,0.510759,0.511487,...,0.498208,0.499368,0.500139,0.500724,0.501312,0.501834,0.502226,0.502604,0.502868,0.503171
181837,0,0.476490,0.476231,0.475933,0.475662,0.475562,0.475498,0.475423,0.475324,0.474942,...,0.480289,0.480393,0.480194,0.479848,0.479471,0.479035,0.478528,0.478001,0.477420,0.476850
166969,1,0.494459,0.494408,0.501106,0.494432,0.498700,0.503248,0.501397,0.497428,0.495933,...,0.425246,0.427500,0.424353,0.420496,0.416421,0.413330,0.420168,0.434886,0.436347,0.424648
79921,1,0.460866,0.461905,0.468009,0.464243,0.466608,0.468762,0.466555,0.463149,0.463525,...,0.335224,0.346610,0.354706,0.362196,0.369487,0.376923,0.389238,0.405741,0.412476,0.408976


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

X = df_balanced_under.iloc[:, 1:]  
y = df_balanced_under.iloc[:, 0] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
print(y_train.unique())

[1 0]


In [9]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [10]:
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# Build the model
model = Sequential([
    (LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], 1))),
    Dropout(0.3),
    (LSTM(100, return_sequences=True)),
    Dropout(0.3),
    (LSTM(100)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# Define callbacks
Early_Stopper = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=5, mode="min")
Checkpoint_Model = tf.keras.callbacks.ModelCheckpoint(monitor="val_accuracy",
                                                      save_best_only=True,
                                                      save_weights_only=True,
                                                      filepath="./modelcheck.weights.h5")

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model
Conv1D_Model = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[Early_Stopper, Checkpoint_Model])

# Evaluate the model
evaluation = model.evaluate(X_test, y_test)


2024-04-25 23:21:17.587684: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-04-25 23:21:17.587735: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-04-25 23:21:17.587740: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-04-25 23:21:17.588072: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-25 23:21:17.588083: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/arthurdesmazures/venv-metal/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using

Epoch 1/100


2024-04-25 23:21:18.269076: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


381/381 ━━━━━━━━━━━━━━━━━━━━ 29s 66ms/step - accuracy: 0.4918 - loss: 0.6942 - val_accuracy: 0.5023 - val_loss: 0.6930
Epoch 2/100
381/381 ━━━━━━━━━━━━━━━━━━━━ 24s 62ms/step - accuracy: 0.4958 - loss: 0.6937 - val_accuracy: 0.5023 - val_loss: 0.6937
Epoch 3/100
381/381 ━━━━━━━━━━━━━━━━━━━━ 24s 62ms/step - accuracy: 0.5066 - loss: 0.6931 - val_accuracy: 0.4977 - val_loss: 0.6932
Epoch 4/100
381/381 ━━━━━━━━━━━━━━━━━━━━ 24s 62ms/step - accuracy: 0.5037 - loss: 0.6932 - val_accuracy: 0.5082 - val_loss: 0.6930
Epoch 5/100
381/381 ━━━━━━━━━━━━━━━━━━━━ 24s 63ms/step - accuracy: 0.4941 - loss: 0.6934 - val_accuracy: 0.4803 - val_loss: 0.6931
Epoch 6/100
381/381 ━━━━━━━━━━━━━━━━━━━━ 24s 64ms/step - accuracy: 0.5082 - loss: 0.6932 - val_accuracy: 0.5023 - val_loss: 0.6929
Epoch 7/100
381/381 ━━━━━━━━━━━━━━━━━━━━ 24s 63ms/step - accuracy: 0.5013 - loss: 0.6935 - val_accuracy: 0.4836 - val_loss: 0.6929
Epoch 8/100
381/381 ━━━━━━━━━━━━━━━━━━━━ 24s 63ms/step - accuracy: 0.4977 - loss: 0.6932 - val_

In [11]:
# Assuming your model is named 'Model'
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype("int32")  # Convert probabilities to binary labels

96/96 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step


In [12]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, log_loss

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Precision
precision = precision_score(y_test, y_pred)
print(f"Precision: {precision:.2f}")

# Recall
recall = recall_score(y_test, y_pred)
print(f"Recall: {recall:.2f}")

# F1 Score
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")

# ROC-AUC Score
roc_auc = roc_auc_score(y_test, y_pred_probs)
print(f"ROC AUC Score: {roc_auc:.2f}")

# Now, you can calculate the log loss
logloss = log_loss(y_test, y_pred_probs)
print(f"Log Loss: {logloss:.2f}")


Confusion Matrix:
 [[1529    0]
 [1515    0]]
Precision: 0.00
Recall: 0.00
F1 Score: 0.00
ROC AUC Score: 0.42
Log Loss: 0.69


/Users/arthurdesmazures/venv-metal/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
